In [ ]:
import argparse
import os
import shutil
import time
import random
#from progress.bar import Bar

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import math
import itertools
import matplotlib.pyplot as plt

import torchvision.datasets as datasets

import datacifar
from alexnet import AlexNet

In [ ]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu" )


# DataLoaders and Dataset

In [ ]:
print('==> Preparing dataset %s' "cifar 100")
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataloader = datacifar.CIFAR100

trainset = dataloader(root='../data', train=True, download=True, transform=transform_train, coarse=True)
trainloader = data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=0)

testset = dataloader(root='../data', train=False, download=False, transform=transform_test, coarse = True)
testloader = data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0)

testloader2 = data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)


# Utils

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value
       Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Test

In [ ]:
def test(testloader, model, criterion):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    model.to(device)
    end = time.time()
    #bar = Bar('Processing', max=len(testloader))
    for batch_idx, (inputs, fine, targets) in enumerate(testloader):
        # measure data loading time
        #print(targets)
        #print( coarse)
        data_time.update(time.time() - end)
        
        
        inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        print( '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=batch_idx + 1,
                    size=len(testloader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=0,
                    eta=0,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    #    bar.next()
   # bar.finish()
    return (losses.avg, top1.avg)

In [ ]:

def train(trainloader, model, criterion, optimizer, epoch, increase = False, coef = 1):
    # switch to train mode
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    #bar = Bar('Processing', max=len(trainloader))
    for batch_idx, (inputs, fine,targets) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)
        #print(targets)
        
        inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        if (increase):
            loss = criterion(outputs, targets) - coef*torch.log(torch.mean(model.intermed.scales()) ) +10
        else:
            loss = criterion(outputs, targets)
        
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if(batch_idx%20 ==0):
            print( '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                        batch=batch_idx + 1,
                        size=len(testloader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=0,
                        eta=0,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                        ))
            print(torch.mean(model_syn.intermed.scales()))

            print(torch.max(model_syn.intermed.scales()))
            print(torch.min(model_syn.intermed.scales()))

    return (losses.avg, top1.avg)



In [ ]:
def test_stoch(testloader, model, criterion):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    model.to(device)
    end = time.time()
    #bar = Bar('Processing', max=len(testloader))
    for batch_idx, (inputs, fine, targets) in enumerate(testloader2):
        # measure data loading time
        #print(targets)
        #print( coarse)
        data_time.update(time.time() - end)
        
        
        inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if(batch_idx%1000 ==0):
            print( '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                        batch=batch_idx + 1,
                        size=len(testloader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=0,
                        eta=0,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                        ))
    #    bar.next()
   # bar.finish()
    return (losses.avg, top1.avg, top5.avg)

In [ ]:
def test_stoch_avg(testloader, model, criterion, iterations = 5):
    top1 =0
    top5 = 0
    for i in range (iterations):
        _, t1, t5 = test_stoch(testloader, model, criterion)
        top1+= t1
        top5+= t5
        
    print(top1/iterations, top5/iterations)

# Noisy network

In [ ]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-5) #-inf


        self.normal = torch.distributions.normal.Normal(0,1)
        self.rhos.requires_grad = True
        self.locs.requires_grad = True
        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self):
        epsilon = self.normal.sample(self.rhos.shape)
        return self.locs + self.scales() * epsilon
                                 
                               
                            
    def forward(self, input):
        noise = self.sample_noise()
        return (input) + noise




# In[61]:


class alexnet_syn(nn.Module):

    def __init__(self, model_features, model_classifier, min_scale,max_scale, given_loc, given_scale):
        super(alexnet_syn, self).__init__()
        
                                
        #self.model_pt1 =  torch.nn.Sequential(*(list(model_features)[0:conv_layers[index]]))
        self.intermed = NoisyActivation( given_loc, given_scale, min_scale, max_scale)
        self.model_pt2 =  torch.nn.Sequential(*(list(model_features)))
        self.model_pt3 = model_classifier
        #self.components = components
        for child in itertools.chain(self.model_pt2, self.model_pt3): 
            for param in child.parameters():
                param.requires_grad = False
            if isinstance(child, nn.modules.batchnorm._BatchNorm):
                child.eval()
                child.affine = False
                child.track_running_stats = False
                
        self.intermed.rhos.reuires_grad = True
        self.intermed.locs.reuires_grad = True
                                 
    def forward(self, img):
                                 
        
        x = self.intermed(img)
        noisy = x.detach()         
        x = self.model_pt2(x)                    
        x = x.view(img.size(0), -1)
        x = self.model_pt3(x)                                 

        return x, noisy
    
         


In [ ]:
model = AlexNet(20)
print(model)
model.load_state_dict(torch.load('old-20class', map_location=torch.device(device)))


In [ ]:
mus = torch.zeros((3,32,32))
scale = torch.ones((3,32,32))*0.01
model_syn = alexnet_syn(model.features, model.classifier ,0.0001,1.0 ,mus, scale )
criterion = nn.CrossEntropyLoss().to(device)
model_syn.load_state_dict(torch.load("old-20class-cloak", map_location=device))

In [ ]:

print(torch.mean(model_syn.intermed.scales()))

print(torch.max(model_syn.intermed.scales()))
print(torch.min(model_syn.intermed.scales()))

In [ ]:
d = list(filter(lambda p: p.requires_grad, model_syn.parameters()))
print(len(d))

In [ ]:
optimizer = optim.Adam(model_syn.parameters(), lr= 0.001) 


In [ ]:
for epoch in range (10):
    train_loss, train_acc = train(trainloader, model_syn, criterion, optimizer, 100, False, 1)

In [ ]:
test_stoch_avg(testloader, model_syn, criterion)

# Mutual information

In [ ]:
for i, (images, fine, labels) in enumerate(testloader2):
    x= images

In [ ]:
import numpy as np
imgs =np.reshape(np.squeeze(images.cpu().detach().numpy()), (1,-1))
lbls = np.reshape(np.squeeze(labels.cpu().detach().numpy()), (1,-1))
encoded_noise = model_syn.intermed.rhos.cpu().detach().numpy()
encoded_original = model_syn.intermed.rhos.cpu().detach().numpy()
image_noisy =np.reshape(np.squeeze(images.cpu().detach().numpy()), (1,-1))
total_correct = 0

model_syn.eval()
model_syn.to(device)
for i, (images, fine, labels) in enumerate(testloader2):
    
    #labels = (labels > 5).long()
    images, labels = images.to(device), labels.to(device)
    
    imgs=np.concatenate((imgs,np.reshape(np.squeeze(images.cpu().detach().numpy()), (1,-1)) ))
    np.save("original-image-mutual_info-20class-input-noise-cloak", imgs)
    lbls=np.concatenate((lbls,np.reshape(np.squeeze(labels.cpu().detach().numpy()), (1,-1)) ))
    np.save("original-labels-mutual_info-20class-input-noise-cloak", lbls)
    
    
    x= images

    x = model_syn.intermed(x)     
  
    
    image_noisy=np.concatenate((image_noisy,np.reshape(np.squeeze(x.cpu().detach().numpy()), (1,-1)) ))
    np.save("noisy-image-mutual_info-20class-input-noise-cloak", image_noisy)

    #print(imgs.shape, lbls.shape, encoded_original.shape, encoded_noise.shape, image_noisy.shape)
    
    x = model_syn.model_pt2(x)                    
    x = x.view(images.size(0), -1)
    output = model_syn.model_pt3(x)                                 


    
    #print(imgs.shape, image_noisy.shape)

    pred = output.detach().max(1)[1]
    total_correct += pred.eq(labels.view_as(pred)).sum()


print('Test Avg. Loss: %f, Accuracy: %f' % (2, float(total_correct) / len(data_test)))
return float(total_correct) / len(data_test)